In [1]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import re
import time
import collections
import os

In [2]:
def build_dataset(words, n_words, atleast=1):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [3]:
with open('english-train', 'r') as fopen:
    text_from = fopen.read().lower().split('\n')[:-1]
with open('vietnam-train', 'r') as fopen:
    text_to = fopen.read().lower().split('\n')[:-1]
print('len from: %d, len to: %d'%(len(text_from), len(text_to)))

len from: 500, len to: 500


In [4]:
text_to[-2:]

['tôi tiếp tục làm thí nghiệm này 1 thời gian',
 'và sau đó , giống các đề tài nghiên cứu hay khác , tôi nhận thêm nguồn tài trợ .']

In [5]:
concat_from = ' '.join(text_from).split()
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])

vocab from size: 1935
Most common words [(',', 564), ('.', 477), ('the', 368), ('and', 286), ('to', 242), ('of', 220)]
Sample data [482, 483, 78, 6, 137, 484, 10, 226, 787, 14] ['rachel', 'pike', ':', 'the', 'science', 'behind', 'a', 'climate', 'headline', 'in']


In [6]:
concat_to = ' '.join(text_to).split()
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab to size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])

vocab to size: 1461
Most common words [(',', 472), ('.', 430), ('tôi', 283), ('và', 230), ('có', 199), ('chúng', 196)]
Sample data [84, 22, 668, 73, 10, 389, 110, 34, 81, 299] ['khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về', 'khí', 'hậu']


In [7]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [8]:
for i in range(len(text_to)):
    text_to[i] += ' EOS'

In [44]:
class Chatbot:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 from_dict_size, to_dict_size, learning_rate, 
                 batch_size, dropout = 0.5, beam_width = 15):
        
        def cell(size, residual, reuse=False):
            c = tf.nn.rnn_cell.GRUCell(size, reuse=reuse)
            if residual:
                c = tf.nn.rnn_cell.ResidualWrapper(c)
            return c
        
        def cells(size, residual = 2):
            cell_list = []
            for i in range(num_layers):
                cell_list.append(cell(size, (i >= num_layers - residual)))
            return cell_list
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        # encoder
        encoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        for n in range(num_layers):
            (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = tf.nn.rnn_cell.MultiRNNCell(cells(size_layer // 2)),
                cell_bw = tf.nn.rnn_cell.MultiRNNCell(cells(size_layer // 2)),
                inputs = encoder_embedded,
                sequence_length = self.X_seq_len,
                dtype = tf.float32,
                scope = 'bidirectional_rnn_%d'%(n))
            encoder_embedded = tf.concat((out_fw, out_bw), 2)
            
        bi_state = tf.concat((state_fw[-1], state_bw[-1]), -1)
        self.encoder_state = tuple([bi_state] * num_layers)
        
        self.encoder_state = tuple(self.encoder_state[-1] for _ in range(num_layers))
        print(self.encoder_state)
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        # decoder
        decoder_embeddings = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        decoder_cells = tf.nn.rnn_cell.MultiRNNCell(cells(size_layer))
        dense_layer = tf.layers.Dense(to_dict_size)
        
        training_helper = tf.contrib.seq2seq.TrainingHelper(
                inputs = tf.nn.embedding_lookup(decoder_embeddings, decoder_input),
                sequence_length = self.Y_seq_len,
                time_major = False)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = decoder_cells,
                helper = training_helper,
                initial_state = self.encoder_state,
                output_layer = dense_layer)
        training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = training_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.Y_seq_len))
        
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                embedding = decoder_embeddings,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS)
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = decoder_cells,
                helper = predicting_helper,
                initial_state = self.encoder_state,
                output_layer = dense_layer)
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = True,
                maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
        self.training_logits = training_decoder_output.rnn_output
        self.predicting_ids = predicting_decoder_output.sample_id
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
size_layer = 256
num_layers = 4
embedded_size = 128
learning_rate = 0.001
batch_size = 16
epoch = 20

In [45]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Chatbot(size_layer, num_layers, embedded_size, len(dictionary_from), 
                len(dictionary_to), learning_rate,batch_size)
sess.run(tf.global_variables_initializer())

(<tf.Tensor 'concat_4:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'concat_4:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'concat_4:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'concat_4:0' shape=(?, 256) dtype=float32>)


In [46]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

In [47]:
X = str_idx(text_from, dictionary_from)
Y = str_idx(text_to, dictionary_to)

In [48]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [49]:
for i in range(epoch):
    total_loss, total_accuracy = 0, 0
    for k in range(0, len(text_to), batch_size):
        index = min(k+batch_size, len(text_to))
        batch_x, seq_x = pad_sentence_batch(X[k: index], PAD)
        batch_y, seq_y = pad_sentence_batch(Y[k: index ], PAD)
        predicted, accuracy,loss, _ = sess.run([model.predicting_ids, 
                                                model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
        total_loss += loss
        total_accuracy += accuracy
    total_loss /= (len(text_to) / batch_size)
    total_accuracy /= (len(text_to) / batch_size)
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1, total_loss, total_accuracy))

epoch: 1, avg loss: 6.534297, avg accuracy: 0.061955
epoch: 2, avg loss: 6.023520, avg accuracy: 0.079758
epoch: 3, avg loss: 5.883783, avg accuracy: 0.083657
epoch: 4, avg loss: 5.692253, avg accuracy: 0.103227
epoch: 5, avg loss: 5.532007, avg accuracy: 0.112095
epoch: 6, avg loss: 5.364343, avg accuracy: 0.116559
epoch: 7, avg loss: 5.218357, avg accuracy: 0.124333
epoch: 8, avg loss: 5.048472, avg accuracy: 0.129905
epoch: 9, avg loss: 4.805836, avg accuracy: 0.142372
epoch: 10, avg loss: 4.560356, avg accuracy: 0.160021
epoch: 11, avg loss: 4.312730, avg accuracy: 0.180998
epoch: 12, avg loss: 4.061545, avg accuracy: 0.211403
epoch: 13, avg loss: 3.797839, avg accuracy: 0.240503
epoch: 14, avg loss: 3.492597, avg accuracy: 0.281195
epoch: 15, avg loss: 3.154245, avg accuracy: 0.332226
epoch: 16, avg loss: 2.872106, avg accuracy: 0.380268
epoch: 17, avg loss: 2.617850, avg accuracy: 0.430409
epoch: 18, avg loss: 2.333587, avg accuracy: 0.484457
epoch: 19, avg loss: 2.110324, avg ac

In [50]:
for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
QUESTION: or , if you had to choose between the last two , which one would you choose ?
REAL ANSWER: sau khi thôi không làm đau mọi người nữa , tôi sẽ hỏi họ bạn có đau không ? đau như thế nào ?
PREDICTED ANSWER: và tôi cố gia , tôi sẽ thực hiện từ thời đó , nhưng nhưng thật thật ra , vì vì tôi phải phải ra , và bé có một sinh sinh sinh sinh học học phải phải phải cộng cộng cộng cộng đồng đồng cộng đồng , tôi có 

row 2
QUESTION: i kept on doing this for a while .
REAL ANSWER: hoặc nếu được chọn giữa 2 kiểu đau cuối , bạn sẽ chọn cái nào ?
PREDICTED ANSWER: và tôi muốn luyện tập cho bạn cách huýt gió khắp . 

row 3
QUESTION: and then , like all good academic projects , i got more funding .
REAL ANSWER: tôi tiếp tục làm thí nghiệm này 1 thời gian
PREDICTED ANSWER: và tôi cố gia , tôi sẽ thực hiện từ thời đó , nhưng nhưng thật thật ra , vì vì tôi phải phải ra , và bé có một sinh sinh sinh sinh học học phải phải phải cộng cộng cộng cộng đồng đồng cộng đồng , tôi có 

row 4
QUESTION: